# Day 19: An Elephant Named Joseph

https://adventofcode.com/2016/day/19

## Part 1

### First solution, a dictionary as a linked list

It works, but it's not fast!

In [14]:
def whiteElephantDict(n):
    neig = {}
    for i in range(1,n):
        neig[i] = (i+1)
    neig[n] = 1
    i = 1
    while len(neig)>1:
        if i in neig.keys():
            ne = neig[i]
            neig[i] = neig[ne]
            del neig[ne]
        i+=1
        if i>n:
            i=1
    return list(neig.keys())[0]

In [16]:
print(whiteElephantDict(3))

3


In [18]:
from timeit import default_timer as timer
start = timer()

print(whiteElephantDict(3014387))

stop = timer()
print("Time elapsed =",int(stop-start),"s")

1834471
Time elapsed = 14 s


## Part 1 

### A better idea!

By observing hoh the list of playing elfs evolves, I realised I can simply pop elements from it according to a simple algorithm with slightly different behavior according to whether the lenght of the list is odd (the last player takes the gift of the first, that gets eliminated) or even (the last player gets eliminated).

In [19]:
def whiteElephantFast(n):
    # inizialize player list
    playing = [i for i in range(1,n+1)]
    while len(playing)>1:
        if len(playing)%2==1: # odd number of players
            # remove "even" players
            playing = playing[0::2]
            # last player gets gifts from first player, so pop first player
            playing.pop(0)
            # I'm left with a even number of players
        if len(playing)%2==0: # even number of players
            # remove "even" players, including last
            playing = playing[0::2]
    return playing[0]

In [23]:
print(whiteElephantFast(3014387))

1834471


## Part 2

In [165]:
def whiteElephant2List(n):
    circle = [i for i in range(1,n+1)]
    while len(circle)>1:
        circle.pop(len(circle)//2)      # remove elf on the other side of the circle
        circle = circle[1:]+circle[0:1] # rotate circle (elf that just played goes to the end)
    return circle[0]

In [166]:
whiteElephant2List(5)

2

### Looking for patterns...

Brute force solution does not work (especially with my implementation!), I'll loow for patterns...

* Winner is last number n for n=3^m
* Winner is k for n=3^m+k for k=(1,...,3^m)
* Winner is k for n=3^m+k for k=(0,...,3^m)

In [182]:
for n in range(1,1000):
    if n-whiteElephant2List(n)==0:
        print(n,n-whiteElephant2List(n))

1 0
3 0
9 0
27 0
81 0
243 0
729 0


In [186]:
for n in range(1,243):
    print(n,whiteElephant2List(n),n-whiteElephant2List(n))

1 1 0
2 1 1
3 3 0
4 1 3
5 2 3
6 3 3
7 5 2
8 7 1
9 9 0
10 1 9
11 2 9
12 3 9
13 4 9
14 5 9
15 6 9
16 7 9
17 8 9
18 9 9
19 11 8
20 13 7
21 15 6
22 17 5
23 19 4
24 21 3
25 23 2
26 25 1
27 27 0
28 1 27
29 2 27
30 3 27
31 4 27
32 5 27
33 6 27
34 7 27
35 8 27
36 9 27
37 10 27
38 11 27
39 12 27
40 13 27
41 14 27
42 15 27
43 16 27
44 17 27
45 18 27
46 19 27
47 20 27
48 21 27
49 22 27
50 23 27
51 24 27
52 25 27
53 26 27
54 27 27
55 29 26
56 31 25
57 33 24
58 35 23
59 37 22
60 39 21
61 41 20
62 43 19
63 45 18
64 47 17
65 49 16
66 51 15
67 53 14
68 55 13
69 57 12
70 59 11
71 61 10
72 63 9
73 65 8
74 67 7
75 69 6
76 71 5
77 73 4
78 75 3
79 77 2
80 79 1
81 81 0
82 1 81
83 2 81
84 3 81
85 4 81
86 5 81
87 6 81
88 7 81
89 8 81
90 9 81
91 10 81
92 11 81
93 12 81
94 13 81
95 14 81
96 15 81
97 16 81
98 17 81
99 18 81
100 19 81
101 20 81
102 21 81
103 22 81
104 23 81
105 24 81
106 25 81
107 26 81
108 27 81
109 28 81
110 29 81
111 30 81
112 31 81
113 32 81
114 33 81
115 34 81
116 35 81
117 36 81
118 37 81
1

In [206]:
def isPowerOfThree(n):
    # 1162261467 = 3^19
    return 1162261467 % n == 0; 

solution = 0
counter = 0
for n in range(9,243):
    if isPowerOfThree(n):
        solution = n
        power3 = n
        counter = 1
    else:
        if counter <= power3:
            solution = counter
            counter += 1
        else:
            solution = counter+1
            counter += 2
    print(n,whiteElephant2List(n),solution,counter,whiteElephant2List(n)==solution)

9 9 9 1 True
10 1 1 2 True
11 2 2 3 True
12 3 3 4 True
13 4 4 5 True
14 5 5 6 True
15 6 6 7 True
16 7 7 8 True
17 8 8 9 True
18 9 9 10 True
19 11 11 12 True
20 13 13 14 True
21 15 15 16 True
22 17 17 18 True
23 19 19 20 True
24 21 21 22 True
25 23 23 24 True
26 25 25 26 True
27 27 27 1 True
28 1 1 2 True
29 2 2 3 True
30 3 3 4 True
31 4 4 5 True
32 5 5 6 True
33 6 6 7 True
34 7 7 8 True
35 8 8 9 True
36 9 9 10 True
37 10 10 11 True
38 11 11 12 True
39 12 12 13 True
40 13 13 14 True
41 14 14 15 True
42 15 15 16 True
43 16 16 17 True
44 17 17 18 True
45 18 18 19 True
46 19 19 20 True
47 20 20 21 True
48 21 21 22 True
49 22 22 23 True
50 23 23 24 True
51 24 24 25 True
52 25 25 26 True
53 26 26 27 True
54 27 27 28 True
55 29 29 30 True
56 31 31 32 True
57 33 33 34 True
58 35 35 36 True
59 37 37 38 True
60 39 39 40 True
61 41 41 42 True
62 43 43 44 True
63 45 45 46 True
64 47 47 48 True
65 49 49 50 True
66 51 51 52 True
67 53 53 54 True
68 55 55 56 True
69 57 57 58 True
70 59 59 60 True
71 

In [217]:
def isPowerOfThree(n):
    return 1162261467 % n == 0; 

def whiteElephant2Fast(N):
    # find closer power of 3 smaller then n
    for m in range(1,19):
        if 3**m < N < 3**(m+1):
            break
    solution = 0
    counter = 0
    # find solution iterating on pattern!
    for n in range(3**m,N+1):
        if isPowerOfThree(n):
            solution = n
            power3 = n
            counter = 1
        else:
            if counter <= power3:
                solution = counter
                counter += 1
            else:
                solution = counter+1
                counter += 2
    return solution

In [218]:
whiteElephant2Fast(239)

235

In [219]:
whiteElephant2Fast(3014387)

1420064